In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import ionpy
from ionpy.analysis import ResultsLoader
from ese.experiment.experiment import CalibrationExperiment
import seaborn as sns
import pandas as pd

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ["CUDA_VISIBLE_DEVICES"] = '3' 

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path = f"{root}/WMH_aug_runs"

dfc = rs.load_configs(
    path,
    properties=False,
)

df = rs.load_metrics(dfc)

In [ ]:
best_exp = rs.get_experiment(
    df=df,
    exp_class=CalibrationExperiment,
    metric="val-dice_score",
    checkpoint="max-val-dice_score",
    device="cuda"
)

In [ ]:
%%yaml dataset_cfg 

_class: ese.experiment.datasets.WMH
annotator: observer_o12
axis: 0
split: val
num_slices: 1
slicing: midslice 
task: Amsterdam 
version: 0.2

In [ ]:
from ionpy.experiment.util import absolute_import
from torch.utils.data import DataLoader
from ionpy.util.torchutils import to_device
import torch
from tqdm.notebook import tqdm

def get_dataset_perf(exp, dataset_config):
    data_cfg = dataset_config.to_dict()
    dataset_cls = absolute_import(data_cfg.pop("_class"))
    Dataset = dataset_cls(**data_cfg)
    dataloader = DataLoader(Dataset, batch_size=1, shuffle=False, drop_last=False)

    items = []
    with torch.no_grad():
        for batch_idx, batch in tqdm(enumerate(dataloader)):
            x, y = to_device(batch, exp.device)
            yhat = exp.model(x)
            loss = exp.loss_func(yhat, y)
            items.append({
                "subj": batch_idx,
                "x": x.cpu().numpy(),
                "y": y.cpu().numpy(),
                "yhat": yhat.detach().cpu().numpy(),
                "loss": loss.item()
            })
    return items

In [ ]:
from ionpy.util import Config
#from ese.experiment.analysis import get_dataset_perf

# val_perf is a dict where each item is the subj id
# with the y, ypred, yloss, ydice
val_perf = get_dataset_perf(
    exp=best_exp, 
    dataset_config=Config(dataset_cfg)
    )